<H3> In this notebooke we actually create the VAE and train on the stock chart images

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

data_input_dir='dataset//'
image_count = 1

#load image_count images as np arrays and convert to torch tensor
dataset = torch.tensor(np.asarray([np.load(data_input_dir+'0-'+str(i)+'.npy') for i in range(image_count)]),dtype=torch.float)
print(dataset.shape)






<h3>Dataloader to create batches and shuffle the data

In [ ]:
batch_size=10
#sampler= torch.utils.data.RandomSampler(dataset)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
class AE(torch.nn.Module):
  def __init__(self):
    super(AE, self).__init__()
    self.encoder = torch.nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=(2, 2), stride=(2),padding=1),
        torch.nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2, 2), stride=(2),padding=1),
        
        nn.Conv2d(16, 8, kernel_size=(2, 2), stride=(1)),
        torch.nn.ReLU(),
        nn.MaxPool2d(kernel_size=(1, 1), stride=(1)),
        
        nn.Conv2d(8, 4, kernel_size=(1, 1), stride=(1)),
        torch.nn.ReLU(),

        nn.MaxPool2d(kernel_size=(1, 1), stride=(1))
      )

    self.decoder = torch.nn.Sequential(        
        nn.Upsample(scale_factor=2, mode='nearest'),
        nn.Conv2d(4, 8, kernel_size=(1, 1), stride=(1, 1),padding=1),
        torch.nn.ReLU(),
        nn.Upsample(scale_factor=2, mode='nearest'),
        nn.Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1)),
        torch.nn.ReLU(),
        nn.Upsample(scale_factor=1, mode='nearest'),
        nn.Conv2d(16, 3, kernel_size=(4, 4), stride=(1, 1))
    )

  def forward(self, x):
    latent = self.encoder(x)
    x_out = self.decoder(latent)
    return x_out

model = AE().to('cpu')

print(dataset.shape)
model(dataset).shape

torch.cuda.empty_cache()

In [ ]:
# Define the loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()
lr=0.02
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=500, gamma=0.05)

loss_list=[]
# Train the model
for epoch in range(5000):
    for idx, (images) in enumerate(data_loader):
        images.to('cpu')
        output = model(images.to('cpu')).to('cpu')
        loss = criterion(output, images.to('cpu'))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        loss_list.append(loss.item())

    # Print loss
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/10], Loss: {loss.item():.4f}')


In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_list)
plt.title('Loss Curve')

In [ ]:
torch.max(output[0])

In [ ]:
import matplotlib.pyplot as plt
for i in range(1):
    plt.figure()
    plt.imshow(output[i][2].to('cpu').detach())

In [ ]:
np.save('test.npy',np.round(torch.sigmoid(output[0,:,:,:].to('cpu').detach())))
np.save('test_real.npy',np.round(torch.sigmoid(images[0,:,:,:].to('cpu').detach())))

In [ ]:

plt.title("Real")
plt.imshow(np.load('test_real.npy').transpose((1,2,0)), interpolation='none', aspect='auto')
plt.show()

plt.title("Rec")
plt.imshow(np.load('test.npy').transpose((1,2,0)), interpolation='none', aspect='auto')
plt.show()


In [ ]:
# testing new pics
t2 = torch.tensor(np.asarray(np.load('dataset/0-123.npy')),dtype=torch.float)
t2_p = np.round(torch.sigmoid(model(t2).to('cpu').detach()))

np.save('test.npy',np.round(torch.sigmoid(t2_p.to('cpu').detach())))
np.save('test_real.npy',np.round(torch.sigmoid(t2.to('cpu').detach())))


plt.title("Real")
plt.imshow(np.load('test_real.npy').transpose((1,2,0)), interpolation='none', aspect='auto')
plt.show()

plt.title("Rec")
plt.imshow(np.load('test.npy').transpose((1,2,0)), interpolation='none', aspect='auto')
plt.show()

In [ ]:
a = torch.nn.Conv2d(3,100,kernel_size=(3),padding='same')
b = torch.nn.ReLU()
c = torch.nn.MaxPool2d((2,2))
d = torch.nn.Conv2d(100,50,kernel_size=(3), padding='same')
e = torch.nn.ReLU()
f = torch.nn.MaxPool2d((2,2))
g = torch.nn.Conv2d(50,50,kernel_size=(3), padding='same')
h = torch.nn.ReLU()

i = torch.nn.MaxPool2d((2,2))


j = torch.nn.Conv2d(50,50,kernel_size=(3),padding='same')
k = torch.nn.ConvTranspose2d(50,100,kernel_size=(3), padding='same')
l = torch.nn.Conv2d(50,50,kernel_size=(3),padding='same')
m = torch.nn.Upsample((2,2),mode='nearest')

print(out_9.shape)
print(out_10.shape)

out_0 = a(dataset[0])
out_1 = b(out_0)
out_2 = c(out_1)
out_3 = d(out_2)
out_4 = e(out_3)
out_5 = f(out_4)
out_6 = g(out_5)
out_7 = h(out_6)
out_8 = i(out_7)
out_9 = j(out_8)
out_10 = k(out_9)
# out_11 = j(out_10)
# out_12 = k(out_11)



print(out_9.shape)
print(out_10.shape)
# print(out_11.shape)
# print(out_12.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(out_0[9,0,:,:].detach())
plt.figure()
plt.imshow(out_1[9,0,:,:].detach())
plt.figure()
plt.imshow(out_2[9,0,:,:].detach())
plt.figure(figsize=(10,10))
#plt.imshow(flat_out_2[0,:].detach())
plt.imshow(out_3.detach())

In [ ]:
plt.figure()
plt.imshow(up_0[0,0,:,:].detach())
plt.figure()
plt.imshow(up_1[0,0,:,:].detach())
plt.figure()
plt.imshow(up_2[0,0,:,:].detach())
plt.figure()
plt.imshow(up_3[0,0,:,:].detach())